In [2]:
from openai import OpenAI
import os
import pandas as pd
from pydantic import BaseModel, Field
from typing import List
from tqdm import tqdm
import json
from random import sample
import jsonlines as jsonl
from openai import OpenAI


OpenAI.organization = "" #Lea's

client = OpenAI(api_key='')


In [3]:
class RelationClassification(BaseModel):
    id: str = Field(description="The ID of the comment being analyzed")    
    label: int = Field(description="The label associated with the argument (1: attacks, 3: no use, 5: supports)") 

In [51]:
def prep_fewshot_samples_3ways(samples_file, n):
    df = pd.read_csv(samples_file)
    ids = df['id'].to_list()
    sampled = sample(ids, n)
    print(sampled)
    df = df[df['id'].isin(sampled)]
    comment = df.iloc[0]['comment_text']
    output = f"Comment: {comment}\n The comment attacks (1), makes no use (3), or supports (5) the following argument(s):\n"
    #print(output)
    for i, row in df.iterrows():
        argument = row['argument_text']
        output = f"{output} Argument {i}: {argument}\n"
        label = row['label']
        print(label)
        if label == 2:
            label = 1
            print(label)
        if label == 4:
            label = 5
            print(label)
        output = f"{output} Label: {label}\n\n"
    return output

In [56]:
def prep_fewshot_samples_5ways(samples_file, n):
    df = pd.read_csv(samples_file)
    ids = df['id'].to_list()
    sampled = sample(ids, n)
    print(sampled)
    df = df[df['id'].isin(sampled)]
    comment = df.iloc[0]['comment_text']
    output = f"Comment: {comment}\n The comment explicitly attacks (1), implicitly attacks (2), makes no use (3), implicitly supports (4), or explicitly supports (5) the following argument(s):\n"
    #print(output)
    for i, row in df.iterrows():
        argument = row['argument_text']
        output = f"{output} Argument {i}: {argument}\n"
        label = row['label']
        output = f"{output} Label: {label}\n\n"
    return output

In [43]:
def classify_text_3ways(id: str, comment_text: str, argument: str, topic: str, samples: str) -> dict:
    completion = client.beta.chat.completions.parse(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": f"""
            Analyze the given comment about {topic} in relation to a specific argument. You need to:
            Identify if the comment makes use of the given argument. Assign the following labels:
            - 1 if the comment attacks the argument.
            - 3 if the comment makes no use of the argument.
            - 5 if the comment supports the argument.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.

            Some examples:
            {samples}
            
            The argument to analyze is: {argument}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "label": "the label for the use of the argument in the comment"
            }}
            
            Analyze the following comment in relation to the given argument:
            """},
            {"role": "user", "content": comment_text},
        ],
        response_format=RelationClassification,
        temperature=0,
        top_p=1,
    )
    return completion.choices[0].message.content

In [57]:
def classify_text_5ways(id: str, comment_text: str, argument: str, topic: str, samples: str) -> dict:
    completion = client.beta.chat.completions.parse(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": f"""
            Analyze the given comment about {topic} in relation to a specific argument. You need to:
            Identify if the comment makes use of the given argument. Assign the following labels:
            - 1 if the comment attacks the argument explicitly.
            - 2 if the comment attacks the argument implicitly/vaguely.
            - 3 if the comment makes no use of the argument.
            - 4 if the comment supports the argument implicitly/vaguely.
            - 5 if the comment supports the argument explicitly.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.

            Some examples:
            {samples}
            
            The argument to analyze is: {argument}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "label": "the label for the use of the argument in the comment"
            }}
            
            Analyze the following comment in relation to the given argument:
            """},
            {"role": "user", "content": comment_text},
        ],
        response_format=RelationClassification,
        temperature=0,
        top_p=1,
    )
    return completion.choices[0].message.content

In [58]:
def process_comments_with_arguments(df: pd.DataFrame, output_file: str, topic: str, samples: str, detailed = False):
    with jsonl.open(output_file, mode='w') as writer:
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing comments"):
            comment_id = row['id']
            comment_text = row['comment_text']
            argument_text = row['argument_text']
            if detailed == False:
                model = classify_text_3ways
            else:
                model = classify_text_5ways
            try:
                gpt_response = model(comment_id, comment_text, argument_text, topic, samples)
                classification = json.loads(gpt_response)
                writer.write(classification)

            except json.JSONDecodeError as e:
                print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
                error_entry = {"id": comment_id, "label": 3}
                writer.write(error_entry)
                continue

            except Exception as e:
                print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
                error_entry = {"id": comment_id, "label": 3}
                writer.write(error_entry)
                continue

In [47]:
gm = pd.read_csv('../../clean_data/GM_structured_main.csv')
samples = prep_fewshot_samples_3ways('../../clean_data/GM_structured_shots.csv', 1)
process_comments_with_arguments(gm, 'comarg_gm_relation_identification3ways_gpt_1shot.jsonl', 'gay marriage', samples, detailed=False)

['11arg5']
2
1


Processing comments: 100%|██████████| 1279/1279 [14:30<00:00,  1.47it/s]


In [60]:
gm = pd.read_csv('../../clean_data/GM_structured_main.csv')
samples = prep_fewshot_samples_5ways('../../clean_data/GM_structured_shots.csv', 5)
process_comments_with_arguments(gm, 'comarg_gm_relation_identification5ways_gpt_5shot.jsonl', 'gay marriage', samples, detailed=True)

['11arg5', '11arg3', '11arg2', '11arg1', '11arg4']


Processing comments: 100%|██████████| 1279/1279 [15:26<00:00,  1.38it/s]


In [65]:
ugip = pd.read_csv('../../clean_data/UGIP_structured_main.csv')
samples = prep_fewshot_samples_3ways('../../clean_data/UGIP_structured_shots.csv', 5)
process_comments_with_arguments(ugip, 'comarg_ugip_relation_identification3way_gpt_5shot.jsonl', 'whether "Under God" should appear in the US Pledge of Allegiance', samples, detailed=False)

['414721783arg5', '414721767arg6', '414721783arg4', '414721767arg5', '414721767arg4']
3
3
3
3
3


Processing comments: 100%|██████████| 1007/1007 [12:23<00:00,  1.35it/s]
